In [ ]:
import numpy as np
import minisom as som 
from pysomvis import PySOMVis

#init synthetic data
x = np.linspace(0,15,500)
y = np.sin(x)
y += np.random.normal(0,.2,len(y))
synth = np.stack((x,y,np.arange(0,len(x)))).T

#train with miniSOM library
s = som.MiniSom(x=20, y=2, input_len=synth.shape[1], sigma=1.0, learning_rate=0.8, random_seed=7)
s.train_random(synth, 100, verbose=False)

#visualization
pysomviz = PySOMVis(weights=s._weights, input_data=synth)
pysomviz._mainview

In [ ]:
import pandas as pd
import numpy as np
from pysomvis import PySOMVis
import minisom as som
import datetime
from sklearn import datasets, preprocessing

data = pd.read_excel('temps.xlsx', index_col=0)
data = data.reset_index()

def calc(row):
    return datetime.datetime(int(row[0]), int(row[1]), int(row[2]))

data['time'] = data.apply(calc, axis=1)
data.drop(data.columns[[0, 1, 2, 5, 7, 9]], axis=1, inplace=True)

data['RH2M'] = data.groupby(['RH2M']).apply(lambda x: x[['RH2M']].interpolate())
data['PS'] = data.groupby(['PS']).apply(lambda x: x[['PS']].interpolate())
data['T2M'] = data.groupby(['T2M']).apply(lambda x: x[['T2M']].interpolate())
data['WS50M'] = data.groupby(['WS50M']).apply(lambda x: x[['WS50M']].interpolate())
data['ALLSKY_SFC_LW_DWN'] = data.groupby(['ALLSKY_SFC_LW_DWN']).apply(lambda x: x[['ALLSKY_SFC_LW_DWN']].interpolate())

data = data.set_index(keys='time')
robustscaler = preprocessing.RobustScaler()
for i in range(data.shape[1]):
    x_scaled = robustscaler.fit_transform(data.iloc[:,i].values.reshape(-1,1))
    data.iloc[:,i] = x_scaled
year = np.array(data.loc['1985':'1986'])

#train with miniSOM library
s2 = som.MiniSom(x=15, y=3, input_len=year.shape[1], sigma=1.0, learning_rate=0.8, random_seed=8)
s2.train_random(year, 10000, verbose=False)

In [ ]:
#np.save('_weights.npy',s2._weights)
np.array(data.loc['1985':'1986'])

In [ ]:
#visualization
pysomviz2 = PySOMVis(weights=s2._weights, input_data=np.array(data.loc['1985':'2020']))
pysomviz2._mainview

In [ ]:
from SOMToolBox_Parse import SOMToolBox_Parse
from pysomvis import PySOMVis
from sklearn import datasets
import holoviews as hv
import numpy as np
from typing import Tuple, List
from visualizations.graph import Graph

iris = datasets.load_iris()

iris_data = SOMToolBox_Parse('datasets\\iris\\iris.vec').read_weight_file()
iris_weights = SOMToolBox_Parse('datasets\\iris\\iris.wgt.gz').read_weight_file()

#visualization
pysomviz3 = PySOMVis(weights=iris_weights['arr'], m=iris_weights['ydim'], n=iris_weights['xdim'],
                                                  dimension=iris_weights['vec_dim'], input_data=iris.data)
pysomviz3._mainview

In [ ]:
import numpy as np

import input_utils
import som_tools

#def som_test(input_image = 'stick_figure.png', input_dataset = "chainlink",
#             som_width = 50, som_height = 50, initial_learning_rate = .1, learning_rate_decay = 10, initial_radius = 1, 
#             radius_decay = 10, initial_radius = 1, radius_decay = 10, n_epochs = 10):
#    image_path = '../mnemonics/'
    active_unit_matrix, distance_matrix = input_utils.load_mnemonic_image(image_path+input_image, som_width, som_height)

    print("reading input")
    #X, y = input_utils.read_dataset("datasets/", "chainlink")
    X, y = input_utils.read_dataset("../datasets/", input_dataset)
    print("reading input done")

    print("generating som")
    som = som_tools.MnemonicSOM(active_unit_matrix, distance_matrix, len(X[0]), 
        initial_learning_rate = initial_learning_rate, 
        learning_rate_decay = learning_rate_decay, 
        initial_radius = initial_radius, 
        radius_decay = radius_decay)
    print("generating som done")

    #som.show_som(X, y)
    #som.show_som2(X)

    print("started training")
    som.train(X, n_epochs)
    print("training done")

    #som.show_som(X, y)
    #som.show_som2(X)

    return som




X, y = input_utils.read_dataset("datasets/", "iris")

som = som_test(initial_learning_rate = 2.5, learning_rate_decay = 5, initial_radius = 2, radius_decay = 10)
#plt.title('radius_decay = 2.5')
som.show_som(X, y)

In [ ]:
def calculate_UMatrix(weights, m, n, dim):
    U = weights.reshape(m, n, dim)
    U = np.insert(U, np.arange(1, n), values=0, axis=1)
    U = np.insert(U, np.arange(1, m), values=0, axis=0)
    #calculate of interpolation
    for i in range(U.shape[0]): 
        if i%2==0:
            for j in range(1,U.shape[1],2):
                U[i,j][0] = np.linalg.norm(U[i,j-1] - U[i,j+1], axis=-1)
        else:
            for j in range(U.shape[1]):
                if j%2==0: 
                    U[i,j][0] = np.linalg.norm(U[i-1,j] - U[i+1,j], axis=-1)
                else:      
                    U[i,j][0] = (np.linalg.norm(U[i-1,j-1] - U[i+1,j+1], axis=-1) + np.linalg.norm(U[i+1,j-1] - U[i-1,j+1], axis=-1))/(2*np.sqrt(2))

    U = np.sum(U, axis=2) #move from Vector to Scalar

    for i in range(0, U.shape[0], 2): #count new values
        for j in range(0, U.shape[1], 2):
            region = []
            if j>0: region.append(U[i][j-1]) #check left border
            if i>0: region.append(U[i-1][j]) #check bottom
            if j<U.shape[1]-1: region.append(U[i][j+1]) #check right border
            if i<U.shape[0]-1: region.append(U[i+1][j]) #check upper border

            U[i,j] = np.median(region)

    return U
def calculate_DMatrix(weights, m, n):

    D=np.zeros(m * n)

    for i in range(m):
        for j in range(n):
            p = i*n+j
            region = []
            if j>0: region.append(p-1)
            if i>0: region.append(p-n)
            if j<n-1: region.append(p+1)
            if i<m-1: region.append(p+n)

            D[p] = np.median(np.linalg.norm(weights[region] - weights[p], axis=-1))      
    return D

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

import input_utils
import som_tools


_m, _n, _dim = 50, 50, 4
active_unit_matrix, distance_matrix = input_utils.load_mnemonic_image("mnemonics/stick_figure.png", 
                                                                      som_width = _m, som_height = _m)

X, y = input_utils.read_dataset("datasets/", 'iris')
som = som_tools.MnemonicSOM(active_unit_matrix, distance_matrix, len(X[0]), initial_learning_rate = .1, 
                            learning_rate_decay = 10, initial_radius = 1, radius_decay = 10)
som.train(X, epochs=10)

ret = []
for x in range(len(som.units)):
    for y in range(len(som.units[0])):
        if som.units[x][y]: ret.append(som.units[x][y].get_weights())
        else:               ret.append(np.full(som.input_lenght, 0))

ret = np.array(ret)

In [ ]:
from SOMToolBox_Parse import SOMToolBox_Parse
from pysomvis import PySOMVis
from sklearn import datasets
import holoviews as hv
import numpy as np
from typing import Tuple, List
from visualizations.graph import Graph

iris = datasets.load_iris()

iris_data = SOMToolBox_Parse('datasets\\iris\\iris.vec').read_weight_file()
iris_weights = SOMToolBox_Parse('datasets\\iris\\iris.wgt.gz').read_weight_file()

#visualization
pysomviz3 = PySOMVis(weights=iris_weights['arr'], m=iris_weights['ydim'], n=iris_weights['xdim'],
                                                  dimension=iris_weights['vec_dim'], input_data=iris.data, 
                                                  classes=iris.target, component_names=iris.target_names)
pysomviz3._mainview

In [2]:
from SOMToolBox_Parse import SOMToolBox_Parse
from pysomvis import PySOMVis
from sklearn import datasets
import holoviews as hv
import numpy as np
from typing import Tuple, List
from visualizations.graph import Graph

dataset = 'iris' #animals, chainlink, iris

_calsses = np.loadtxt('datasets\\'+dataset+'\\'+dataset+'.txt', delimiter=',')[:,1]
_data = SOMToolBox_Parse('datasets\\'+dataset+'\\'+dataset+'.vec').read_weight_file()
_weights = SOMToolBox_Parse('datasets\\'+dataset+'\\'+dataset+'.wgt.gz').read_weight_file()


pysomviz3 = PySOMVis(weights=_weights['arr'], m=_weights['ydim'], n=_weights['xdim'], dimension=_weights['vec_dim'],
                     input_data=_data['arr'], classes=_calsses, component_names=np.unique(_calsses).astype(str))
pysomviz3._mainview

Column
    [0] Column
        [0] Column
            [0] Row(margin=5, width=700)
                [0] StaticText(value='<b></b>')
                [1] Select(options=OrderedDict([('Component P...]), value=0)
                [2] Select(options=OrderedDict([('PiYG', ...]), value='jet')
                [3] Button(name='↶')
                [4] Button(name='↷')
                [5] Button(name='↔')
                [6] Button(name='↕')
                [7] Checkbox(name='interpolation')
        [1] Row
            [0] Column
                [0] HoloViews(DynamicMap)
            [1] Row
                [0] Column
                    [0] Column(margin=5, name='Component Planes', width=300)
                        [0] StaticText(value='<b>Component Planes</b>')
                        [1] IntSlider(end=3, name='0.0')
                    [1] Str(str)
    [1] Column
        [0] Row()

In [ ]:
pysomviz3._visualizations[-1]._calculate()